# Modules and Classes

In [1]:
import dataclasses
import datetime as dt
from pathlib import Path
import sys

# Make climate_paper_v2 files importable
CURRENT_DIR = Path.cwd()
CLIMATE_PAPER_DIR = CURRENT_DIR.parent / 'src' / 'climate_paper_v2'
sys.path.insert(0, str(CLIMATE_PAPER_DIR.absolute()))

## Modules

Modules are Python files that can be imported.  Importing a module executes the Python in the file.

In [2]:
import climate_paper_figs as cpf

Hello from c:\Users\jostev\github\python-improvers-2\src\climate_paper_v2\climate_paper_figs.py body!


In [3]:
# We can access variables and functions
print(f"DATA_DIR defined in module: {cpf.DATA_DIR}")
station_file_path = Path('some_stationdata.txt')
print(f"Station name from file path: {cpf.get_station_name(station_file_path)}")

DATA_DIR defined in module: c:\Users\jostev\github\python-improvers-2\data
Station name from file path: some_station


## Classes for data

Classes are another way to group variables (attributes) and functions (methods).  Everything is a class.
`dir` shows the attributes and methods that are present.  The private "__dunder__" methods determine
class behaviour within Python.

In [4]:
my_integer = 1
print(f"type: {type(my_integer)}")
dir(my_integer)

type: <class 'int'>


['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_count',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'is_integer',
 

In [5]:
# A simple class needs a function (__init__) to initialise it.
# A __repr__ function defines how it is printed.
# The methods of the class instance take "self" as the first parameter
# which allows them to access internal data.

class MyClass:
    def __init__(self, attribute: int):
        self.attribute = attribute
    
    def __repr__(self):
        return f"MyClass({self.attribute})"

my_class = MyClass(1)
print(f"my class: {my_class}")
print(f"type: {type(my_class)}")
dir(my_class)

my class: MyClass(1)
type: <class '__main__.MyClass'>


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'attribute']

In [6]:
# Dataclasses are a convenient way to store data with a __repr__ and __init__
# and with type hints to help keep your code right.
# This example is similar to a row from a database table.

@dataclasses.dataclass
class MontoringStation:
    sensor_type: str
    sensor_serial_number: int
    longitude: float
    latitude: float
    deploy_date: dt.date

station = MontoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    longitude=-3.1,
    latitude=54.0,
    deploy_date=dt.date(2023, 1, 20)
)

print(station)

# Attributes are accessed and updated using . notation
print(f"serial number: {station.sensor_serial_number}")
station.sensor_serial_number = 5678
print(f"new serial number: {station.sensor_serial_number}")

# Dataclasses can conveniently be turned into dictionaries or tuples
print(f"asdict: {dataclasses.asdict(station)}")

MontoringStation(sensor_type='Guralp 2000', sensor_serial_number=1234, longitude=-3.1, latitude=54.0, deploy_date=datetime.date(2023, 1, 20))
serial number: 1234
new serial number: 5678
asdict: {'sensor_type': 'Guralp 2000', 'sensor_serial_number': 5678, 'longitude': -3.1, 'latitude': 54.0, 'deploy_date': datetime.date(2023, 1, 20)}


In [7]:
# Classes can have methods that act on their attributes.

@dataclasses.dataclass
class MontoringStation:
    sensor_type: str
    sensor_serial_number: int
    longitude: float
    latitude: float
    deploy_date: dt.date

    def uptime(self) -> int:
        """Return time since deploy date in days"""
        delta = dt.date.today() - self.deploy_date
        return delta.days

station = MontoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    longitude=-3.1,
    latitude=54.0,
    deploy_date=dt.date(2023, 1, 20)
)

print(f"uptime: {station.uptime()} days")

uptime: 733 days


In [8]:
# Classes can keep a record of past states with custom updates.
@dataclasses.dataclass
class MontoringStation:
    sensor_type: str
    sensor_serial_number: int
    longitude: float
    latitude: float
    deploy_date: dt.date

    def __post_init__(self):
        # Register
        self.previous_deployments: list[tuple[float, float, dt.date]] = []
        self.deploy(self.longitude, self.latitude, self.deploy_date)

    def uptime(self) -> int:
        """Return time since deploy date in days"""
        delta = dt.date.today() - self.deploy_date
        return delta.days

    def deploy(
        self, longitude: float, latitude: float, deploy_date: dt.date = dt.date.today()
    ):
        """Record a new deployment of the sensor."""
        # Store old deployment
        current_deployment = (self.longitude, self.latitude, self.deploy_date)
        self.previous_deployments.append(current_deployment)

        # Update current attributes
        self.longitude = longitude
        self.latitude = latitude
        self.deploy_date = deploy_date
    
    # A property is a method that takes only "self" as an argument and
    # behaves like an (read only in this case) attribute.
    @property
    def deployments_count(self):
        return len(self.previous_deployments)


station = MontoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    latitude=54.0,
    longitude=-3.1,
    deploy_date=dt.date(2023, 1, 20)
)

station.deploy(52, -2, dt.date(2024, 1, 12))
station.deploy(63, -14, dt.date(2024, 6, 2))
print(f"deployments count: {station.deployments_count}")
print(f"previous deployments: {station.previous_deployments}")

deployments count: 3
previous deployments: [(-3.1, 54.0, datetime.date(2023, 1, 20)), (-3.1, 54.0, datetime.date(2023, 1, 20)), (52, -2, datetime.date(2024, 1, 12))]


In [9]:
# Inheritance allows classes to reuse code.
@dataclasses.dataclass
class SeismicMonitoringStation(MontoringStation):
    high_pass_filter_frequency: float


@dataclasses.dataclass
class GeoMagMonitoringStation(MontoringStation):
    magnet_type: str


seismic_station = SeismicMonitoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    latitude=54.0,
    longitude=-3.1,
    deploy_date=dt.date(1993, 1, 20),
    high_pass_filter_frequency=45.3
)

geomag_station = GeoMagMonitoringStation(
    sensor_type="MagTastic SuperPro",
    sensor_serial_number=9999,
    latitude=54.0,
    longitude=-3.1,
    deploy_date=dt.date(2003, 3, 5),
    magnet_type="Inductive Pickup"
)

print(f"seismic uptime: {seismic_station.uptime()} days")
print(f"geomag deployments: {geomag_station.deployments_count}")

seismic uptime: 11690 days
geomag deployments: 1


### Exercise

+ Create a MonitoringStationCatalog class.  Internally, it should store a private list of stations with a public-facing `register` method that adds in a new station. It should have a public method for `get_station_count`
+ Add in the stations above.
+ Add a `locations()` function that returns a dictionary of data, with serial_number as the key and a tuple of (longitude, latitude) as the value.
+ Note what happens to the locations() result when you redeploy any of the stations.
+ BONUS: add an `unregister` function that removes a station based on the serial number.